### Hyperparameter Tuning Using Grid Search With Parallel Computing

**Important Imports**

In [ ]:
import random
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error
import time
import joblib
import threading
import multiprocessing
import os

**Generate Dataset**

In [ ]:
def generate_data(xmin,xmax,Delta,noise):
    # Calculate f=sin(x1)+cos(x2)
    x1 = np.arange(xmin,xmax+Delta,Delta)   # generate x1 values from xmin to xmax
    x2 = np.arange(xmin,xmax+Delta,Delta)   # generate x2 values from xmin to xmax
    x1, x2 = np.meshgrid(x1,x2)             # make x1,x2 grid of points
    f = np.sin(x1) + np.cos(x2)             # calculate for all (x1,x2) grid
    # Add random noise to f
    random.seed(2020)                       # set random seed for reproducibility
    for i in range(len(f)):
        for j in range(len(f[0])):
            f[i][j] = f[i][j] + random.uniform(-noise,noise)  # add random noise to f(x1,x2)

    # Calculate and print the number of records
    num_records = x1.size  # size of the grid (number of points)
    print(f"Number of records generated: {num_records}")
    return x1,x2,f

# Create {x1,x2,f} dataset every 1.0 from -10 to 10, with a noise of +/- 0.5
x1,x2,f=generate_data(-10,10,0.4,0.5) # Changed delta to generate different number of records

**Prepare Data for Model Training**

In [ ]:
def prepare_data(x1,x2,f):
    X = []
    for i in range(len(f)):
        for j in range(len(f)):
            X_term = []
            X_term.append(x1[i][j])
            X_term.append(x2[i][j])
            X.append(X_term)
    y=f.flatten()
    X=np.array(X)
    y=np.array(y)
    return X,y

# Prepare X and y
X,y = prepare_data(x1,x2,f)

**Kernel Ridge Regression (KRR) with Cross-Validation**

*BASELINE*

In [ ]:
def KRR_function(X,y):
    # Initialize lists with final results
    y_pred_total = []
    y_test_total = []
    # Split data into test and train: random state fixed for reproducibility
    kf = KFold(n_splits=10,shuffle=True,random_state=2020)
    # kf-fold cross-validation loop
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # Scale X_train and X_test
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        # Fit KRR with (X_train_scaled, y_train), and predict X_test_scaled
        KRR = KernelRidge()
        y_pred = KRR.fit(X_train_scaled, y_train).predict(X_test_scaled)
        # Append y_pred and y_test values of this k-fold step to list with total values
        y_pred_total.append(y_pred)
        y_test_total.append(y_test)
    # Flatten lists with test and predicted values
    y_pred_total = [item for sublist in y_pred_total for item in sublist]
    y_test_total = [item for sublist in y_test_total for item in sublist]
    # Calculate error metric of test and predicted values: rmse
    rmse = np.sqrt(mean_squared_error(y_test_total, y_pred_total))
    return rmse
KRR_function(X,y)

*Hyperparameter Grid Search for KRR*

In [ ]:
def KRR_function(hyperparams,X,y):
    # Assign hyper-parameters
    alpha_value,gamma_value = hyperparams
    # Initialize lists with final results
    y_pred_total = []
    y_test_total = []
    # Split data into test and train: random state fixed for reproducibility
    kf = KFold(n_splits=10,shuffle=True,random_state=2020)
    # kf-fold cross-validation loop
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # Scale X_train and X_test
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        # Fit KRR with (X_train_scaled, y_train), and predict X_test_scaled
        KRR = KernelRidge(kernel='rbf',alpha=alpha_value,gamma=gamma_value)
        y_pred = KRR.fit(X_train_scaled, y_train).predict(X_test_scaled)
        # Append y_pred and y_test values of this k-fold step to list with total values
        y_pred_total.append(y_pred)
        y_test_total.append(y_test)
    # Flatten lists with test and predicted values
    y_pred_total = [item for sublist in y_pred_total for item in sublist]
    y_test_total = [item for sublist in y_test_total for item in sublist]
    # Calculate error metric of test and predicted values: rmse
    rmse = np.sqrt(mean_squared_error(y_test_total, y_pred_total))
    print('KRR k-fold cross-validation . alpha: %7.6f, gamma: %7.4f, RMSE: %7.4f' %(alpha_value,gamma_value,rmse))
    return rmse

**Sequential Code Benchmarking**

In [ ]:
graph_x = []
graph_y = []
graph_z = []

counter = 0

def create_hyperparams_grid(X,y):
    global counter
    for alpha_value in np.arange(-5.0,2.0,0.7):
        alpha_value = pow(10,alpha_value)
        graph_x_row = []
        graph_y_row = []
        graph_z_row = []
        for gamma_value in np.arange(0.0,20,2):
            counter += 1
            print(f"Task {counter} is running")
            hyperparams = (alpha_value,gamma_value)
            rmse = KRR_function(hyperparams,X,y)
            graph_x_row.append(alpha_value)
            graph_y_row.append(gamma_value)
            graph_z_row.append(rmse)
        graph_x.append(graph_x_row)
        graph_y.append(graph_y_row)
        graph_z.append(graph_z_row)

start = time.time()
create_hyperparams_grid(X,y)
end = time.time()
print("Execution time:", end-start)
print("Value of counter:", counter)

graph_x=np.array(graph_x)
graph_y=np.array(graph_y)
graph_z=np.array(graph_z)
min_z=np.min(graph_z)
pos_min_z=np.argwhere(graph_z == np.min(graph_z))[0]
print('Minimum RMSE: %.4f' %(min_z))
print('Optimum alpha: %f' %(graph_x[pos_min_z[0],pos_min_z[1]]))
print('Optimum gamma: %f' %(graph_y[pos_min_z[0],pos_min_z[1]]))

**Parallel Code Benchmarking**

*Parallelizes the sequential section with - lock -*

*Multithreading approach:*

In [ ]:
import concurrent.futures

lock = threading.Lock()
graph_x = []
graph_y = []
graph_z = []
counter = 0
alpha_values = np.arange(-5.0,2.0,0.7)
gamma_values = np.arange(0.0,20,2)

def func_for_alpha_loop(alpha_value):
    global counter
    alpha_value = pow(10,alpha_value)
    graph_x_row = []
    graph_y_row = []
    graph_z_row = []
    for gamma_value in gamma_values:
        with lock:
            counter = counter + 1
        print(f"Task {counter} is running in thread {threading.get_ident()}")
        print(f"Task {counter} is running in process {os.getpid()}")
        hyperparams = (alpha_value,gamma_value)
        rmse = KRR_function(hyperparams,X,y)
        graph_x_row.append(alpha_value)
        graph_y_row.append(gamma_value)
        graph_z_row.append(rmse)
    with lock:
        graph_x.append(graph_x_row)
        graph_y.append(graph_y_row)
        graph_z.append(graph_z_row)
    
start = time.time() 
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    executor.map(func_for_alpha_loop, alpha_values) 
end = time.time()
print("Execution time:", end-start)
print("Value of counter: ", counter)

graph_x=np.array(graph_x)
graph_y=np.array(graph_y)
graph_z=np.array(graph_z)
min_z=np.min(graph_z)
pos_min_z=np.argwhere(graph_z == np.min(graph_z))[0]

print('Minimum RMSE: %.4f' %(min_z))
print('Optimum alpha: %f' %(graph_x[pos_min_z[0],pos_min_z[1]]))
print('Optimum gamma: %f' %(graph_y[pos_min_z[0],pos_min_z[1]]))

*Multiprocessing approach:*

In [ ]:
manager = multiprocessing.Manager()
counter = manager.Value('i', 0)
lock = manager.Lock()
graph_x = manager.list()
graph_y = manager.list()
graph_z = manager.list()
alpha_values = np.arange(-5.0,2.0,0.7)
gamma_values = np.arange(0.0,20,2)

def func_for_alpha_loop(alpha_value):  
    alpha_value = pow(10,alpha_value)
    graph_x_row = []
    graph_y_row = []
    graph_z_row = []
    for gamma_value in gamma_values:
        hyperparams = (alpha_value,gamma_value)
        rmse = KRR_function(hyperparams,X,y)
        with lock:
            counter.value = counter.value + 1 
        print(f"Task {counter.value} is running in process {os.getpid()}")
        graph_x_row.append(alpha_value)
        graph_y_row.append(gamma_value)
        graph_z_row.append(rmse)
    with lock:
        graph_x.append(graph_x_row)
        graph_y.append(graph_y_row)
        graph_z.append(graph_z_row)

start = time.time()
joblib.Parallel(n_jobs=8)(joblib.delayed(func_for_alpha_loop)(alpha_value) for alpha_value in alpha_values)
end = time.time()
print("Execution time:", end-start)
print("counter: ", counter.value)

graph_x=np.array(graph_x)
graph_y=np.array(graph_y)
graph_z=np.array(graph_z)
min_z=np.min(graph_z)
pos_min_z=np.argwhere(graph_z == np.min(graph_z))[0]
print('Minimum RMSE: %.4f' %(min_z))
print('Optimum alpha: %f' %(graph_x[pos_min_z[0],pos_min_z[1]]))
print('Optimum gamma: %f' %(graph_y[pos_min_z[0],pos_min_z[1]]))

**References:**

- https://towardsdatascience.com/grid-search-in-python-from-scratch-hyperparameter-tuning-3cca8443727b

- https://joblib.readthedocs.io/en/stable/parallel.html

- https://realpython.com/intro-to-python-threading/

- https://superfastpython.com/thread-safe-write-to-file-in-python/

### End